In [28]:
import pandas as pd
import numpy as np
import os
import requests
from datetime import datetime, timedelta
from datetime import date
import holidays
import holidays_es
from holidays_es.holidays import get_provinces, Province
from holidays import Spain
import requests
from bs4 import BeautifulSoup

In [9]:
# Define column names and data types
column_names = ['year', 'month', 'day_of_month', 'day_of_week', 'customers', 'revenue_day', 'revenue_night',
                'weather_sunny', 'weather_rainy', 'weather_cloudy', 'weather_temperature', 'holiday', 'cruise_ship']

column_types = {'year': 'int', 'month': 'int', 'day_of_month': 'int', 'day_of_week': 'int', 'customers': 'int',
                'revenue_day': 'float', 'revenue_night': 'float', 'weather_sunny': 'int', 'weather_rainy': 'int',
                'weather_cloudy': 'int', 'weather_temperature': 'int', 'holiday': 'int', 'cruise_ship': 'int'}




In [21]:

# Set the path of the parent directory
parent_dir = 'C:/Users/germa/github/rest_in_bcn_extras/control'

# Initialize an empty DataFrame
la_lluna_01 = pd.DataFrame()

# Loop over the year folders
for year_folder in os.listdir(parent_dir):
    year_path = os.path.join(parent_dir, year_folder)
    
    # Loop over the month folders
    for month_folder in os.listdir(year_path):
        month_path = os.path.join(year_path, month_folder)
        
        # Get the month from the name of the folder
        month = month_folder[:3]
        
        # Convert month to month number
        if month.startswith('EN'):
            month_num = 1
        elif month.startswith('FE'):
            month_num = 2
        elif month.startswith('MAR'):
            month_num = 3
        elif month.startswith('AB'):
            month_num = 4
        elif month.startswith('MAY'):
            month_num = 5
        elif month.startswith('JUN'):
            month_num = 6
        elif month.startswith('JUL'):
            month_num = 7
        elif month.startswith('AG'):
            month_num = 8
        elif month.startswith('SE'):
            month_num = 9
        elif month.startswith('OC'):
            month_num = 10
        elif month.startswith('NO'):
            month_num = 11
        elif month.startswith('D'):
            month_num = 12
        else:
            month_num = None
        
        # Loop over the Excel files
        for file in os.listdir(month_path):
            
            # Check if the file starts with 'REC'
            if file.startswith('REC'):
                file_path = os.path.join(month_path, file)
                
                # Read the Excel file
                df = pd.read_excel(file_path, header=None)
                
                # Get the day of the month and daily revenue
                day_of_month = df.iloc[2:8, [1, 3, 5, 7, 9, 11, 13]].stack().reset_index(drop=True)
                daily_revenue = df.iloc[2:8, [2, 4, 6, 8, 10, 12, 14]].stack().reset_index(drop=True)
                
                # Create a DataFrame with day_of_month and daily_revenue
                df_new = pd.DataFrame({'day_of_month': day_of_month, 'daily_revenue': daily_revenue})
                
                # Add the month column as the month number
                df_new['month'] = month_num
                
                # Add the year column
                df_new['year'] = year_folder
                
                # Add the DataFrame to la_lluna_01
                la_lluna_01 = pd.concat([la_lluna_01, df_new], ignore_index=True)
print(la_lluna_01)


     day_of_month daily_revenue  month  year
0               0             0      8  2020
1               0             0      8  2020
2               0             0      8  2020
3               1             0      8  2020
4               2        322.37      8  2020
...           ...           ...    ...   ...
1300            0             0      3  2023
1301            0             0      3  2023
1302            0             0      3  2023
1303            0             0      3  2023
1304            0           NaN      3  2023

[1305 rows x 4 columns]


In [22]:
la_lluna_01.sample(10)

,day_of_month,daily_revenue,month,year
1261,29,0,2,2023
633,25,606.41,5,2021
53,19,886.65,12,2020
1263,0,0,2,2023
1021,12,1524.75,3,2022
785,0,0,4,2022
419,30,0,12,2021
366,10,0,8,2021
569,2,179.73,3,2021
1288,19,0,3,2023


In [23]:
la_lluna_01.to_csv("la_lluna_01.csv", index=False)

In [24]:
la_lluna_01[['year', 'month', 'day_of_month']].applymap(lambda x: x == '').sum()


year            0
month           0
day_of_month    0
dtype: int64

In [25]:
la_lluna_01.dropna(subset=['year', 'month', 'day_of_month'], inplace=True)

In [28]:
la_lluna_01.shape

(1292, 4)

In [29]:
la_lluna_01.head()

,day_of_month,daily_revenue,month,year
0,0,0,8,2020
1,0,0,8,2020
2,0,0,8,2020
3,1,0,8,2020
4,2,322.37,8,2020


In [30]:
la_lluna_01.dtypes


day_of_month     object
daily_revenue    object
month             int64
year             object
dtype: object

In [31]:
la_lluna=pd.read_csv("G:/My Drive/IRONHACK/FINAL PROJECT/la_lluna_01.csv")

In [34]:
la_lluna.shape

(1305, 4)

In [35]:
la_lluna.sample(5)

,day_of_month,daily_revenue,month,year
669,30,0.00,11,2021
1209,10,743.75,1,2023
393,4,1922.40,12,2021
395,6,2513.25,12,2021
611,3,0.00,5,2021


In [36]:
la_lluna.dtypes

day_of_month      object
daily_revenue    float64
month              int64
year               int64
dtype: object

In [37]:

# create a new column with the full date
la_lluna['date'] = la_lluna['day_of_month'].map(str) + '-' + la_lluna['month'].map(str) + '-' + la_lluna['year'].map(str)





In [38]:


def try_parse_date(date_string):
    try:
        return pd.to_datetime(date_string, format='%d-%m-%Y')
    except ValueError:
        return pd.NaT

la_lluna['date'] = la_lluna.apply(lambda row: try_parse_date(f"{row['day_of_month']}-{row['month']}-{row['year']}"), axis=1)



In [39]:
la_lluna.shape

(1305, 5)

In [40]:
# Create 'weekday_str' column with weekday names
la_lluna['weekday_str'] = la_lluna['date'].dt.strftime('%A')

# Create 'weekday' column with weekday numbers (Monday=1, Sunday=7)
la_lluna['weekday'] = la_lluna['date'].dt.dayofweek + 1



In [43]:
la_lluna['weekday'] = la_lluna['weekday'].fillna(0).astype('Int64')


In [45]:
la_lluna.to_csv("la_lluna.csv", index=False)

In [46]:

la_lluna['closed'] = np.where(la_lluna['daily_revenue'] == 0, 1, 0)


In [47]:
la_lluna['closed'] = la_lluna['closed'].fillna(0).astype('Int64')

In [48]:
la_lluna.isna().sum()

day_of_month      13
daily_revenue      7
month              0
year               0
date             204
weekday_str      204
weekday            0
closed             0
dtype: int64

In [49]:
la_lluna['date'].dropna(inplace=True)

In [ ]:
la_lluna = la_lluna['month'].

In [ ]:
la_lluna['day_of_month'] = la_lluna['day_of_month'].astype('Int64')
la_lluna['year'] = la_lluna['year'].astype('Int64')
la_lluna['month'] = la_lluna['month'].astype('Int64')


In [ ]:
la_lluna = la_lluna.loc[~((la_lluna['month'] == 2) & (la_lluna['day_of_month'] > 28) & (la_lluna['year'] != 2020))]


In [3]:
la_lluna = pd.read_csv("la_lluna (1).csv")

In [33]:
la_lluna.dtypes

day_of_month              int64
daily_revenue           float64
month                     int64
year                      int64
date             datetime64[ns]
weekday_str              object
weekday                   int64
closed                    int64
holidays                  int64
feelslike               float64
precip                  float64
windgust                float64
windspeed               float64
cloudcover              float64
moonphase               float64
dtype: object

In [5]:

provinces = get_provinces()
print(provinces)



['alava', 'albacete', 'alicante', 'almeria', 'asturias', 'avila', 'badajoz', 'baleares', 'barcelona', 'bilbao', 'burgos', 'caceres', 'cadiz', 'cantabria', 'castellon', 'ceuta', 'ciudad-real', 'cordoba', 'la-coruna', 'cuenca', 'gijon', 'girona', 'granada', 'guadalajara', 'guipuzcoa', 'huelva', 'huesca', 'jaen', 'leon', 'lleida', 'logrono', 'lugo', 'madrid', 'malaga', 'melilla', 'murcia', 'navarra', 'ourense', 'oviedo', 'palencia', 'palma-de-mallorca', 'las-palmas', 'pamplona', 'pontevedra', 'la-rioja', 'salamanca', 'san-sebastian', 'santander', 'segovia', 'sevilla', 'soria', 'tarragona', 'tenerife', 'teruel', 'toledo', 'valencia', 'valladolid', 'vitoria', 'vizcaya', 'zamora', 'zaragoza']


In [101]:
Province(name="barcelona", year=2022).local_holidays()

[datetime.date(2022, 9, 24), datetime.date(2022, 9, 26)]

In [102]:
Province(name="barcelona", year=2022).regional_holidays()

[datetime.date(2022, 4, 18),
 datetime.date(2022, 6, 6),
 datetime.date(2022, 6, 24),
 datetime.date(2022, 12, 26)]

In [9]:
holidays_list = [(2023, 1, 6),
                (2023, 4, 7),
                (2023, 5, 1),
                (2023, 8, 15),
                (2023, 10, 12),
                (2023, 11, 1),
                (2023, 12, 6),
                (2023, 12, 8),
                (2023, 12, 25),
                (2023, 5, 29), 
                (2023, 9, 25),
                (2023, 4, 10),
                (2023, 6, 24),
                (2023, 9, 11),
                (2023, 12, 26),
                (2020, 1, 1),
                (2020, 1, 6),
                (2020, 4, 10),
                (2020, 5, 1),
                (2020, 8, 15),
                (2020, 10, 12),
                (2020, 12, 8),
                (2020, 12, 25),
                (2020, 6, 1),
                (2020, 9, 24),
                (2020, 4, 13),
                (2020, 6, 24),
                (2020, 9, 11),
                (2020, 12, 26),
                (2021, 1, 1),
                (2021, 1, 6),
                (2021, 4, 2),
                (2021, 5, 1),
                (2021, 10, 12),
                (2021, 11, 1),
                (2021, 12, 6),
                (2021, 12, 8),
                (2021, 12, 25),
                (2021, 5, 24),
                (2021, 9, 24),
                (2021, 4, 5),
                (2021, 6, 24),
                (2021, 9, 11),
                (2022, 1, 1),
                (2022, 1, 6),
                (2022, 4, 15),
                (2022, 8, 15),
                (2022, 10, 12),
                (2022, 11, 1),
                (2022, 12, 6),
                (2022, 12, 8),
                (2022, 9, 24),
                (2022, 9, 26),
                (2022, 4, 18),
                (2022, 6, 6),
                (2022, 6, 24),
                (2022, 12, 26)]

In [10]:


# Convert 'date' column to datetime format
la_lluna['date'] = pd.to_datetime(la_lluna['date'])

# Create new 'holidays' column and set all values to 0
la_lluna['holidays'] = 0

# Loop over every date in the holidays_list and set 'holidays' to 1 for matching dates in 'la_lluna'
for holiday in holidays_list:
    holiday_date = pd.to_datetime('-'.join(map(str, holiday)))
    if holiday_date in la_lluna['date'].values:
        la_lluna.loc[la_lluna['date'] == holiday_date, 'holidays'] = 1
    else:
        # if the holiday is not in the dataframe, you could choose to add a new row with NaN values
        # or ignore the holiday
        print(f"Holiday {holiday_date} not in dataframe")


Holiday 2023-04-07 00:00:00 not in dataframe
Holiday 2023-05-01 00:00:00 not in dataframe
Holiday 2023-08-15 00:00:00 not in dataframe
Holiday 2023-10-12 00:00:00 not in dataframe
Holiday 2023-11-01 00:00:00 not in dataframe
Holiday 2023-12-06 00:00:00 not in dataframe
Holiday 2023-12-08 00:00:00 not in dataframe
Holiday 2023-12-25 00:00:00 not in dataframe
Holiday 2023-05-29 00:00:00 not in dataframe
Holiday 2023-09-25 00:00:00 not in dataframe
Holiday 2023-04-10 00:00:00 not in dataframe
Holiday 2023-06-24 00:00:00 not in dataframe
Holiday 2023-09-11 00:00:00 not in dataframe
Holiday 2023-12-26 00:00:00 not in dataframe
Holiday 2020-04-10 00:00:00 not in dataframe
Holiday 2020-05-01 00:00:00 not in dataframe
Holiday 2020-06-01 00:00:00 not in dataframe
Holiday 2020-04-13 00:00:00 not in dataframe
Holiday 2020-06-24 00:00:00 not in dataframe


In [20]:
la_lluna.sample(5)

,day_of_month,daily_revenue,month,year,date,weekday_str,weekday,closed,holidays
305,30,0.00,1,2021,2021-01-30,Saturday,6,1,0
211,30,0.00,10,2020,2020-10-30,Friday,5,1,0
162,11,790.70,9,2020,2020-09-11,Friday,5,0,1
530,8,651.39,9,2021,2021-09-08,Wednesday,3,0,0
578,26,493.89,10,2021,2021-10-26,Tuesday,2,0,0


In [21]:
la_lluna.to_csv("la_lluna.csv", index=False)

In [22]:

path = 'G:/My Drive/IRONHACK/FINAL PROJECT/'
filename = 'weather_BCN_20200101_20230309.csv'
columns_to_keep = ['datetime', 'feelslikemax', 'feelslikemin', 'feelslike', 'precip', 'windgust', 'windspeed', 'cloudcover', 'moonphase', 'description', 'conditions']

weather = pd.read_csv(path + filename, usecols=columns_to_keep)
weather = weather.sort_values(by='datetime', ascending=True)


In [23]:
weather.sample(5)

,datetime,feelslikemax,feelslikemin,feelslike,precip,windgust,windspeed,cloudcover,moonphase,conditions,description
323,2020-11-19,21.5,7.8,14.9,0.000,17.1,16.5,25.7,0.14,Partially cloudy,Becoming cloudy in the afternoon.
694,2021-11-25,12.9,7.2,10.7,11.903,51.3,35.9,61.8,0.70,"Rain, Partially cloudy",Partly cloudy throughout the day with a chance...
992,2022-09-19,25.9,17.6,21.9,0.000,31.7,23.5,41.1,0.79,Partially cloudy,Clearing in the afternoon.
543,2021-06-27,27.9,20.9,23.6,0.000,21.6,17.5,27.4,0.59,Partially cloudy,Partly cloudy throughout the day.
889,2022-06-08,27.7,19.8,22.8,0.028,21.2,21.1,35.1,0.30,"Rain, Partially cloudy",Partly cloudy throughout the day with late aft...


In [24]:
weather.to_csv("weather.csv", index=False)

In [25]:
for index, row in weather.iterrows():
    # find the corresponding row in la_lluna dataframe
    la_lluna_row = la_lluna[la_lluna['date'] == row['datetime']]
    # if the corresponding row is not found, skip this row
    if len(la_lluna_row) == 0:
        continue
    # update the columns in la_lluna dataframe
    la_lluna.loc[la_lluna_row.index, 'feelslike'] = row['feelslike']
    la_lluna.loc[la_lluna_row.index, 'precip'] = row['precip']
    la_lluna.loc[la_lluna_row.index, 'windgust'] = row['windgust']
    la_lluna.loc[la_lluna_row.index, 'windspeed'] = row['windspeed']
    la_lluna.loc[la_lluna_row.index, 'cloudcover'] = row['cloudcover']
    la_lluna.loc[la_lluna_row.index, 'moonphase'] = row['moonphase']


In [27]:
la_lluna.sample(5)

,day_of_month,daily_revenue,month,year,date,weekday_str,weekday,closed,holidays,feelslike,precip,windgust,windspeed,cloudcover,moonphase
142,21,580.17,8,2020,2020-08-21,Friday,5,0,0,29.9,0.000,21.1,18.1,29.8,0.08
475,15,492.17,7,2021,2021-07-15,Thursday,4,0,0,23.3,1.024,31.8,24.0,37.1,0.19
16,17,1033.68,1,2020,2020-01-17,Friday,5,0,0,12.5,0.000,48.5,28.3,24.6,0.75
111,21,479.71,7,2020,2020-07-21,Tuesday,2,0,0,26.6,0.000,19.4,15.1,12.9,0.02
971,16,823.56,11,2022,2022-11-16,Wednesday,3,0,0,17.8,0.000,39.6,23.9,32.9,0.75


In [30]:
import requests
from bs4 import BeautifulSoup

url = 'https://www.cruisemapper.com/ports/barcelona-port-82' # Replace with the URL of the website you're trying to scrape
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')
tables = soup.find_all('table')

for i, table in enumerate(tables):
    print(f"Table {i}:")
    print(table)


In [31]:
response

<Response [403]>